This project is greatly inspired by the following article:

[Building a full-text search engine in 150 lines of Python code](https://bart.degoe.de/building-a-full-text-search-engine-150-lines-of-code/)

and the Information Retrieval and Web Search lectures at the University of Mannheim, Germany

# WARNING Rewrite positional index and phrase retrieval

In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
len(inverted_index_sortpost.index)

99311

In [2]:
%%time
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

from search import bool_retrieval, phrase_retrieval, vsm, bim, bim_extension, query_likelihood
from saveload import save_load

CPU times: user 1.13 s, sys: 315 ms, total: 1.45 s
Wall time: 6.5 s


# Indexing the documents collection

In [3]:
%%time
inverted_index_sortpost,inverted_index_unsortpost,positional_index_sortpost,positional_index_unsortpost,term_freqs_log,\
term_freqs_raw, doc_freqs, unigram=save_load(save=True, save_path="./data")

.....................Inverted Index with sorted postings.....................
200 document(s) have been indexed
400 document(s) have been indexed
600 document(s) have been indexed
800 document(s) have been indexed
1000 document(s) have been indexed
1200 document(s) have been indexed
1400 document(s) have been indexed
.....................Inverted Index with unsorted postings.....................
200 document(s) have been indexed
400 document(s) have been indexed
600 document(s) have been indexed
800 document(s) have been indexed
1000 document(s) have been indexed
1200 document(s) have been indexed
1400 document(s) have been indexed
.....................Positional Index with sorted postings.....................
200 document(s) have been indexed
400 document(s) have been indexed
600 document(s) have been indexed
800 document(s) have been indexed
1000 document(s) have been indexed
1200 document(s) have been indexed
1400 document(s) have been indexed
.....................Positional Index w

In [4]:
inverted_index_sortpost.index["experiment"][:6], len(inverted_index_sortpost.index)

([1, 11, 12, 16, 17, 19], 5853)

'experiment' appears in docs 1, 11, 12, 16, 17, 19

The collection is made of 5853 preprocessed terms

In [5]:
inverted_index_sortpost.t_char_index['experimental']

{'ent', 'eri', 'exp', 'ime', 'men', 'nta', 'per', 'rim', 'tal', 'xpe'}

'experimental' is made of the above character k-grams

In [6]:
print(inverted_index_sortpost.char_t_index["tal"], len(inverted_index_sortpost.char_t_index["tal"]))

{'determinantal', 'installations', 'experimental-failure', 'complete-compressor-stall', 'abrupt-stall', 'parietale', 'incidental', 'metallic', 'total', 'metal', 'wing-stall', 'fundamentals', 'studies-experimental', 'horizontally', 'stall', 'horizontal', 'stalled', 'noncatalytic', 'static-to-total', 'total-head', 'environmental', 'etal', 'incremental-normal-force', 'stall-naca', 'experimental', 'transcendental', 'detrimental', 'catalytically', 'wing-stalling', 'digital', 'total-angle', 'talbot', 'intercrystalline', 'convection-fundamental', 'vital', 'polycrystalline', 'destalling', 'digital-computer', 'orbital', 'stall-limit', 'stalling', 'elemental', 'bimetallic', 'fundamental', 'installation', 'frontal', 'total-tail-assembly', 'catalytic', 'metals', 'liquid-metal', 'fundamentally', 'pivotal', 'stall-flutter', 'horizontal-tail', 'accidental', 'experimentally', 'incremental', 'centripetal', 'total-pressure'} 59


59 terms in the collection include the character 3-gram 'tal'.

In [7]:
positional_index_sortpost.index["metal"]

[12,
 {144: [1, [35]]},
 {268: [2, [12, 69]]},
 {640: [1, [21]]},
 {763: [1, [51]]},
 {856: [1, [23]]},
 {865: [1, [4]]},
 {962: [1, [180]]},
 {1015: [2, [35, 39]]},
 {1027: [2, [14, 18]]},
 {1028: [3, [12, 21, 68]]},
 {1097: [1, [62]]},
 {1349: [2, [66, 72]]}]

The preprocessed term 'metal' appears 12 documents: in document 144 once at positions 35, in document 1015 twice at postions 35 and 39.

# Searching

## Simple boolean retrieval

In [8]:
%%time
query='Yes we can !!!'
bool_retrieval(inverted_index_sortpost, query)

Retrieving document(s) including at least one of the query terms (in their preprocessed form) ...
Your query does not match the documents, please try a new one
CPU times: user 3.08 ms, sys: 0 ns, total: 3.08 ms
Wall time: 7.76 ms


[]

In [9]:
%%time
query='Yes we can !!!'
bool_retrieval(inverted_index_unsortpost, query)

Retrieving document(s) including at least one of the query terms (in their preprocessed form) ...
Your query does not match the documents, please try a new one
CPU times: user 2.07 ms, sys: 0 ns, total: 2.07 ms
Wall time: 6.47 ms


[]

In [10]:
%%time
query='Aerodynamics motors'
bool_retrieval(inverted_index_sortpost, query)

Retrieving document(s) including at least one of the query terms (in their preprocessed form) ...
CPU times: user 1.65 ms, sys: 594 µs, total: 2.25 ms
Wall time: 6.49 ms


[Document(ID=77, content='a comparative analysis of the performance of long range hypervelocity vehicles .   long-range hypervelocity vehicles are studied in terms of their motion in powered flight, and their motion and aerodynamic heating in unpowered flight .  powered flight is analyzed for an idealized propulsion system which rather closely approaches present-day rocket motors . unpowered flight is characterized by a return to earth along a ballistic, skip, or glide trajectory .  only those trajectories are treated which yield the maximum range for a given velocity at the end of powered flight . aerodynamic heating is treated in a manner similar to that employed previously by the senior authors in studying ballistic missiles (naca tn ), with the exception that radiant as well as convective heat transfer is considered in connection with glide and skip vehicles .   the ballistic vehicle is found to be the least efficient of the several types studied in the sense that it generally requ

In [11]:
%%time
query='Aerodynamics motors'
bool_retrieval(inverted_index_sortpost, query, 'AND')

Retrieving document(s) including all the query terms (in their preprocessed form) ...
CPU times: user 1.4 ms, sys: 506 µs, total: 1.91 ms
Wall time: 6.01 ms


[Document(ID=77, content='a comparative analysis of the performance of long range hypervelocity vehicles .   long-range hypervelocity vehicles are studied in terms of their motion in powered flight, and their motion and aerodynamic heating in unpowered flight .  powered flight is analyzed for an idealized propulsion system which rather closely approaches present-day rocket motors . unpowered flight is characterized by a return to earth along a ballistic, skip, or glide trajectory .  only those trajectories are treated which yield the maximum range for a given velocity at the end of powered flight . aerodynamic heating is treated in a manner similar to that employed previously by the senior authors in studying ballistic missiles (naca tn ), with the exception that radiant as well as convective heat transfer is considered in connection with glide and skip vehicles .   the ballistic vehicle is found to be the least efficient of the several types studied in the sense that it generally requ

## Phrase queries retrieval

In [12]:
%%time
#query='''transition studies and skin friction measurements'''
query='''the effects of humidity on the mach number and static pressure in the
working section were investigated and the results are compared with
theoretical estimates at a nominal mach number 2.0'''
phrase_retrieval(positional_index_sortpost, query)

CPU times: user 6.18 ms, sys: 726 µs, total: 6.9 ms
Wall time: 9.54 ms


[Document(ID=466, content='development of the vapour screen method of flow visualization in the ft tunnel at rae bedford.   the vapour screen method of flow visualisation in supersonic wind tunnels is outlined, and the development of a suitable technique for use in the  ft tunnel described, together with the associated optical and photographic equipment .   the results of tests to determine the humidity required to produce an optimum density of fog in the working section over the mach number range temperature discussed .  numerous vapour screen photographs of the flow over and behind delta wings are included and some comparisons made with the corresponding surface oil-flow patterns .   the process of condensation, the physical and optical properties of the resulting fog, and the formation of the vapour screen picture are all considered in some detail .   the effects of humidity on the mach number and static pressure in the working section were investigated and the results are compared 

## Vector Space Model

In [13]:
%%time
query='Bla Bla bla'
vsm(query, positional_index_sortpost, term_freqs_log, doc_freqs, top=3)

CPU times: user 19.8 ms, sys: 0 ns, total: 19.8 ms
Wall time: 19.1 ms


[]

In [14]:
%%time
query="Thermodynamics is useful in aerodynamics"
vsm(query, positional_index_sortpost, term_freqs_log, doc_freqs, top=3)

CPU times: user 17 ms, sys: 4.28 ms, total: 21.3 ms
Wall time: 20.5 ms


[(Document(ID=396, content="variational and lagrangian thermodynamics of thermal convection-fundamental shortcomings of the heat transfer coefficient .   extension of previous analyses, indicating the possibility of extending the thermodynamics of irreversible processes to systems which are not in the vicinity of an equilibrium state and for which onsager's relations are not verified .  this involves generalizations beyond the narrow field of heat transfer and to principles of wider range than those of current nonequilibrium thermodynamics ."),
  'score = 0.6688725638908491'),
 (Document(ID=405, content='tables of thermal properties of gases . tables of thermodynamic and transport properties of air, argon, carbon dioxide, carbon monoxide, hydrogen, nitrogen, oxygen, and steam .'),
  'score = 0.6626044991511518'),
 (Document(ID=1011, content='free-flight measurements of the static and dynamic .   charts have been prepared relating the thermodynamic properties of air in chemical equilibr

In [15]:
query='''what similarity laws must be obeyed when constructing aeroelastic models
of heated high speed aircraft'''
vsm(query, positional_index_sortpost, term_freqs_log, doc_freqs, top=3)

[(Document(ID=573, content='viscous hypersonic similitude .   an extension of classical hypersonic similitude is developed which takes into account the interaction effect of the displacement thickness of the boundary layer .  a basic result of this viscous similitude is that the total drag including frictional drag obeys the classical similarity law for the pressure drag . additional similarity conditions governing viscous effects must be imposed in this similitude .   underlying the similitude is a new hypersonic boundary-layer independence principle .  according to this principle, the principal part of a hypersonic boundary layer with given pressure and wall temperature distributions and free-stream total enthalpy is independent of the (high) external mach number distribution outside the boundary layer .   various features of viscous hypersonic similitudes are discussed .  it is found, for example, that it applies to three- dimensional boundary-layer interaction effects on flat bodie

## Binary Independence models

In [16]:
%%time
query="Thermodynamics is useful in aerodynamics"
bim(query, inverted_index_sortpost, top=3)

CPU times: user 2.29 ms, sys: 828 µs, total: 3.12 ms
Wall time: 2.85 ms


[(Document(ID=1314, content='production of high temperature gases in shock tubes .   this paper is intended to set forth aerodynamic and thermodynamic calculations which are useful in the production of strong shock waves .  the experimental production of strong shock waves is discussed . comparison of the experimental shock strengths with the theoretical calcualtions is made, and finally, some preliminary results of shock tube studies in high temperature gases (up to ,k) are briefly surveyed .'),
  'score = 2.0941132169500674'),
 (Document(ID=1335, content='use of freon- as a fluid for aerodynamic testing .   the thermodynamic properties of freon- have been investigated to determine the possibilities of the use of this gas as a fluid for aerodynamic testing .  the values of velocity of sound in freon-, which are less than one-half those in air, are presented as functions of temperatures and pressure, including measurements at room temperature .  the density of freon- is about four time

In [17]:
%%time
bim(query, inverted_index_unsortpost, top=3)

CPU times: user 2.88 ms, sys: 0 ns, total: 2.88 ms
Wall time: 2.71 ms


[(Document(ID=1314, content='production of high temperature gases in shock tubes .   this paper is intended to set forth aerodynamic and thermodynamic calculations which are useful in the production of strong shock waves .  the experimental production of strong shock waves is discussed . comparison of the experimental shock strengths with the theoretical calcualtions is made, and finally, some preliminary results of shock tube studies in high temperature gases (up to ,k) are briefly surveyed .'),
  'score = 2.0941132169500674'),
 (Document(ID=1335, content='use of freon- as a fluid for aerodynamic testing .   the thermodynamic properties of freon- have been investigated to determine the possibilities of the use of this gas as a fluid for aerodynamic testing .  the values of velocity of sound in freon-, which are less than one-half those in air, are presented as functions of temperatures and pressure, including measurements at room temperature .  the density of freon- is about four time

In [18]:
query='''what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft'''
bim(query, inverted_index_unsortpost, top=3)

[(Document(ID=573, content='viscous hypersonic similitude .   an extension of classical hypersonic similitude is developed which takes into account the interaction effect of the displacement thickness of the boundary layer .  a basic result of this viscous similitude is that the total drag including frictional drag obeys the classical similarity law for the pressure drag . additional similarity conditions governing viscous effects must be imposed in this similitude .   underlying the similitude is a new hypersonic boundary-layer independence principle .  according to this principle, the principal part of a hypersonic boundary layer with given pressure and wall temperature distributions and free-stream total enthalpy is independent of the (high) external mach number distribution outside the boundary layer .   various features of viscous hypersonic similitudes are discussed .  it is found, for example, that it applies to three- dimensional boundary-layer interaction effects on flat bodie

In [19]:
%%time
bim_extension(query, positional_index_sortpost, term_freqs_log, extension='two_poisson', top=3)

CPU times: user 8.21 ms, sys: 0 ns, total: 8.21 ms
Wall time: 7.48 ms


[(Document(ID=329, content='various aerodynamic characteristics in hypersonic rarefied gas flow .   this paper considers the problem of calculating viscous aerodynamic characteristics of blunt bodies at hypersonic speeds and at sufficiently high altitudes where the appropriate mean free path becomes too large for the use of familiar boundary-layer theory but not so large that free molecule concepts apply .   results of an order-of-magnitude analysis are presented to define the regimes of rarefied gas flow and the limits of continuum theory .  based on theoretical and experimental evidence, the complete navier-stokes equations are used as a model, except /very close/ to the free molecule condition .  this model may not necessarily give the shock wave structure in detail but satisfies overall conservation laws and should give a reasonably accurate picture of all mean aerodynamic quantities .   in this /intermediate/ regime there are two fundamental classes of problems ..  a /viscous laye

In [20]:
%%time
bim_extension(query, positional_index_sortpost, term_freqs_log, extension='bm11', top=3)

CPU times: user 5.07 ms, sys: 2.96 ms, total: 8.03 ms
Wall time: 7.63 ms


[(Document(ID=51, content='theory of aircraft structural models subjected to aerodynamic heating and external loads .   the problem of investigating the simultaneous effects of transient aerodynamic heating and external loads on aircraft structures for the purpose of determining the ability of the structure to withstand flight to supersonic speeds is studied .  by dimensional analyses it is shown that .. constructed of the same materials as the aircraft will be thermally similar to the aircraft with respect to the flow of heat through the structure will be similar to those of the aircraft when the structural model is constructed at the same temperature as the aircraft . external loads will be similar to those of the aircraft . subjected to heating and cooling that correctly simulate the aerodynamic heating of the aircraft, except with respect to angular velocities and angular accelerations, without requiring determination of the heat flux at each point on the surface and its variation 

In [21]:
%%time
bim_extension(query, positional_index_sortpost, term_freqs_log, extension='bm25', top=3)

CPU times: user 10.9 ms, sys: 0 ns, total: 10.9 ms
Wall time: 10.5 ms


[(Document(ID=51, content='theory of aircraft structural models subjected to aerodynamic heating and external loads .   the problem of investigating the simultaneous effects of transient aerodynamic heating and external loads on aircraft structures for the purpose of determining the ability of the structure to withstand flight to supersonic speeds is studied .  by dimensional analyses it is shown that .. constructed of the same materials as the aircraft will be thermally similar to the aircraft with respect to the flow of heat through the structure will be similar to those of the aircraft when the structural model is constructed at the same temperature as the aircraft . external loads will be similar to those of the aircraft . subjected to heating and cooling that correctly simulate the aerodynamic heating of the aircraft, except with respect to angular velocities and angular accelerations, without requiring determination of the heat flux at each point on the surface and its variation 

## Language model

In [22]:
%%time
query_likelihood(query, inverted_index_sortpost, unigram, top=3)

CPU times: user 2.28 ms, sys: 870 µs, total: 3.15 ms
Wall time: 3.05 ms


[(Document(ID=5, content='one-dimensional transient heat conduction into a double-layer slab subjected to a linear heat input for a small time internal .   analytic solutions are presented for the transient heat conduction in composite slabs exposed at one surface to a triangular heat rate .  this type of heating rate may occur, for example, during aerodynamic heating .'),
  'score = 0.17142857142857143'),
 (Document(ID=158, content='temperature charts for induction and constant temperature heating .   charts are presented for determining complete temperature historics in spheres, cylinders, and plates .  it is shown that for values of the dimensionless time ratio x greater than . the heating equations reduce to such a simple form that for each shape two charts which give temperatures at any position within the heated or cooled bodies can be plotted .  it is also shown that the usual simple heating and cooling charts can also be used for the determination of temperatures and heating ti